Notebook con todas las funciones desarrolladas en 02.Modelo_Temp_Base para ser importado en los notebooks correspondientes a las diferentes pruebas

In [7]:
def find_events(data, edificios, n_obs_antes, n_obs_despues, variables):
    #DataFrame vacío para la salida
    datos_all_rooms = pd.DataFrame()
    #import pandas as pd
    #Para que no haya problemas de slice y .loc
    pd.options.mode.chained_assignment = None
    
    #Cogemos los datos, y seleccionamos las habitaciones de cada edificio 
    datos = data
    edificio = edificios
    mask = datos['block'].isin(edificio)
    datos = datos.loc[mask]
    rooms = datos['room'].unique()
    
    #Iteramos sobre cada una de las habitaciones
    for room in rooms:
        mask = datos['room'].isin([room])
        datos_room = datos.loc[mask]
        if variables == 1:
            datos_room = datos_room[['V2','V4']]
        elif variables == 2:
            datos_room = datos_room.drop(columns=['dif_cons','cons_total','block','room', 'V12','V26','V5_0','V5_1','V5_2', 'Hora_1', 'Hora_2', 'Hora_3', 'Season_1', 'Season_2', 'Season_3', 'Season_4'],axis=1)
        elif variables == 3:
            datos_room = datos_room.drop(columns=['dif_cons','cons_total','block','room', 'V12','V26','V5_0','V5_1','V5_2'],axis=1)
        else: 
            print('var_mode error')
            break
        #Ponemos la columna de V4 como string y buscamos una secuencia de un 1 seguido de n 0's
        #El número de 0's lo establece n_obs_despues
        text = datos_room.V4.fillna(2).astype(int).astype(str).str.cat()
        time = n_obs_despues*10 #Esto nos da el tiempo en min. Ej: n_obs_despues = 6 -> time = 60'

        sequence = list(np.repeat(str(0),time/10))
        sequence.insert(0,'1'*n_obs_antes)
        sequence= ''.join(sequence)
        pattern = f'(?:{sequence})'
        findings = re.finditer(pattern, text)
        events = [(m.start(0)) for m in findings]  

        df = pd.DataFrame()
        #Para cada una de las secuencias de 1 y 0's, hacemos la selección de las instancias correspondientes
        #del conjunto de datos y las añadimos a un df, que iremos completando con cada uno de los eventos
        for i in range(len(events)):
            ref_inicio = events[i]+n_obs_antes #+1 porque la referencia de inicio es el primer 0 y events empieza en el 1
            df1 = datos_room.iloc[range(ref_inicio - n_obs_antes, ref_inicio + n_obs_despues),:]
            data = df1
            #Una vez que tenemos los eventos quitamos la columna de V4
            #data = data.drop(columns='V4')
            #n_vars = 1 if type(data) is list else data.shape[1]
            df = pd.concat([df,data])
        datos_all_rooms = pd.concat([datos_all_rooms, df])
    return datos_all_rooms

In [8]:
def feature_selection(num_features, datos):
    data = datos
    select = SelectKBest(score_func=f_regression, k=num_features)
    z = select.fit_transform(data.iloc[:,1:], data.iloc[:,0]) 
    filter = select.get_support()
    features = np.array(data.iloc[:,1:].columns.values)
    features = features[filter].tolist()
    features.insert(0, 'V2')
    return features

In [12]:
def shuffle_df(datos, interval):
    df=datos
    arr_3D = df.values.reshape(-1,interval,df.shape[1])
    shuffle_idx = np.random.permutation(arr_3D.shape[0])
    arr_3D = arr_3D[shuffle_idx]
    shuffled_df = pd.DataFrame(arr_3D.reshape(df.shape[0],df.shape[1]))
    return shuffled_df

In [13]:
def split_dataset(data, per_train, per_test, interval, n_obs_antes, n_obs_despues):
    df=data
    train = df.values[0:int((interval*np.ceil(df.shape[0]/interval*per_train)))]
    test =  df.values[int((interval*np.ceil(df.shape[0]/interval*per_train))):int((interval*np.ceil(df.shape[0]/interval*(per_train+per_test))))]
    val = df.values[int((interval*np.ceil(df.shape[0]/interval*(per_train+per_test)))):]
    #Escalado en train y test por separado. Si no se hace aquí ya no se puede porque todo lo que sigue es con arrays 3D
    scaler_y = StandardScaler()
    train_y = scaler_y.fit_transform(train[:,0].reshape(-1, 1))
    val_y = scaler_y.fit_transform(val[:,0].reshape(-1, 1))
    test_y = scaler_y.transform(test[:,0].reshape(-1, 1))
    scaler_x = StandardScaler()
    train_x = scaler_x.fit_transform(train[:,1:]) 
    val_x = scaler_x.fit_transform(val[:,1:]) 
    test_x = scaler_x.transform(test[:,1:])
    
    #Rehacemos los DataFrames con los datos escalados para poder hacer el split
    train = pd.concat([pd.DataFrame(train_y),pd.DataFrame(train_x)], axis=1)
    test = pd.concat([pd.DataFrame(test_y),pd.DataFrame(test_x)], axis=1)
    val = pd.concat([pd.DataFrame(val_y),pd.DataFrame(val_x)], axis=1)

    train = np.array(split(train, len(train)/(n_obs_antes+n_obs_despues)))
    test = np.array(split(test, len(test)/(n_obs_antes+n_obs_despues)))
    val = np.array(split(val, len(val)/(n_obs_antes+n_obs_despues)))
    return train, test, val, scaler_x, scaler_y

In [ ]:
def to_supervised(train, n_input, n_out):
    # Ponemos los datos en 2D: una entrada para cada muestra y paso de tiempo con las n características
    data = train.reshape((train.shape[0]*train.shape[1], train.shape[2])) 
    X, y = list(), list()
    in_start = 0
    # Pasamos por todo el conjunto de datos, de step en step
    for _ in range(len(data)):
        # Definimos cuando empieza el input, cuando acaba, e igual para el output
        # En este caso el output empieza cuando acaba el input
        in_end = in_start + n_input
        out_end = in_end + n_out
        # Iteramos mientras haya ejemplos
        if out_end <= len(data):
            X.append(data[in_start:in_end, :]) # X son todas las variables en t
            y.append(data[in_end:out_end, 0])  # y es la temperatura en t+1 (si n_out=1)
        # Pasamos al siguiente instante de tiempo
        in_start += 1
    X = array(X)
    y = array(y)
    return X, y

In [11]:
def fix_samples(n_obs_despues, n_obs_antes, train_x, test_x, val_x, train_y, test_y, val_y):
    rows_to_keep = n_obs_despues 
    rows_to_drop = n_obs_antes
    period = rows_to_keep + rows_to_drop
    
    train_sel = (np.arange(len(train_x)) % period) < rows_to_keep
    test_sel = (np.arange(len(test_x)) % period) < rows_to_keep
    val_sel = (np.arange(len(val_x)) % period) < rows_to_keep
    
    train_x = train_x[train_sel,:,:]
    train_y = train_y[train_sel,:] 
    test_x = test_x[test_sel,:,:]
    test_y = test_y[test_sel,:] 
    val_x = val_x[val_sel,:,:]
    val_y = val_y[val_sel,:] 
    return train_x, train_y, test_x, test_y, val_x, val_y

In [ ]:
def build_model(hp, n_timesteps, n_features, n_outputs):
    #Esta estructura hay que retocarla
    model = Sequential()
    #Numero de nodos en la primera capa 
    hp_units = hp.Choice('input_unit',[32,64,128])
    model.add(LSTM(units = hp_units, input_shape=(n_timesteps, n_features)))
    model.add(RepeatVector(n_outputs))   
    #Numero de capas ocultas
    for i in range(hp.Int('n_layers', 1, 2)):
        units_i = hp.Choice(f'dense_{i}_units',[32,64,128])
        model.add(LSTM(units = hp_units, return_sequences=True))
    #Dropout
    hp_dropout = hp.Float('Dropout_rate',min_value=0,max_value=0.4,step=0.1)
    model.add(Dropout(hp_dropout))
    model.add(TimeDistributed(Dense(units = hp_units)))
    
    model.add(TimeDistributed(Dense(n_outputs)))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(loss='mae', optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate))
    return model

In [ ]:
def predict(datos_test_x, datos_test_y, n_predict, n_obs_despues, model, n_timesteps, n_features, n_input):
    test_y = datos_test_y
    test_x = datos_test_x
    #Creamos una lista vacía para las predicciones
    predictions = list()
    #Ponemos los índices i y j a 0
    i=0
    #j=0
    #Indice para hacer predicciones en un horizonte de tiempo concreto
    #Saltamos n filas cada m. Ej: tenemos n_obs_despues=12 pero queremos predecir solo la hora siguiente así
    #que predecimos 6 filas, saltamos otras 6 (que serían la 2a hora) y repetimos
    n = n_obs_despues - n_predict 
    m = n_predict   #Salto de n cada m
    test_index = [j for k in range(0,len(test_x), n+m) for j in range(k, m+k) if j<len(test_x)]
    for i in test_index:
        data = test_x[i,:,:]
        # Cogemos como input los valores del instante de tiempo anterior al que vamos a predecir
        # Recordar que el x lleva un lag de 1 respecto a la y, por eso es el elemento i y no el i-1
        # Le damos la forma que el modelo necesita para predecir
        input_x = data.reshape((1, n_timesteps, n_features))
        
        # Prediccion para cada paso de tiempo
        yhat = model.predict(input_x, verbose=0)
        yhat_sequence = yhat[0] 

        # Guardamos las predicciones
        predictions.append(yhat_sequence)
        
        # Reescribimos el valor de la temperatura del siguiente paso con la prediccion
        # Añadimos una condición más para que en el último paso de cada evento no actualice, ya
        # que reescribiría datos de otro caso
        if i < len(test_x) and (i not in range(n_predict-1,len(test_x), n_obs_despues)):
            try: test_x[i+1,n_input,0] = yhat[0]
            except IndexError: continue
            #La excepción es porque al llegar al final no hay un i+1 que actualizar y salta un IndexError
        #else: break
        #j+=1
        
    predictions = array(predictions)
    #Seleccionamos solo los ejemplos del conjunto de test que vamos a predecir
    test_y = test_y[test_index,:]
    return predictions, test_y

In [ ]:
def metrics(test_y, prediction):
    rmse = np.sqrt(mean_squared_error(test_y,prediction))
    cvrmse = rmse/np.mean(test_y)*100
    r2 = r2_score(test_y, prediction)
    return cvrmse, r2, rmse

In [1]:
from platform import python_version

print(python_version())

3.8.3
